# Face Recognition using Transfer Learning on VGG16

## Creating images dataset using opencv through webcam

In [20]:
import cv2
import numpy as np

# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

# Initialize Webcam
cap = cv2.VideoCapture(0)
count = 0

# Collect 1000 samples of your face from webcam input
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = 'C://Users//vivekPC//Desktop//MLOps WS//FRTL//FRsamples//vivek' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 1000: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found


## Loading the VGG16 Model and freezing all layers except the top 4

In [21]:
from keras.applications import VGG16

# VGG16 was designed to work on 224 x 224 pixel input images sizes
img_rows = 224
img_cols = 224 

# loads the VGG16 model without the top or FC layers
model = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in model.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


## Making a function that returns our Fully Connected Head

In [22]:
def addTopModel(bottom_model, num_classes, D=256):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""
    top_model = bottom_model.output
    top_model = Flatten(name = "flatten")(top_model)
    top_model = Dense(D, activation = "relu")(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation = "softmax")(top_model)
    return top_model

model.input

<tf.Tensor 'input_2:0' shape=(None, 224, 224, 3) dtype=float32>

## Getting model layers

In [23]:
model.layers

## Adding our FC Head back onto VGG16 and get summary

In [24]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 2

FC_Head = addTopModel(model, num_classes)

modelnew = Model(inputs=model.input, outputs=FC_Head)

print(modelnew.summary())

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

## Loading our face sample Dataset

In [25]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'FRsamples/train/'
validation_data_dir = 'FRsamples/validation/'

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 16
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 1600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


## Training our top layers and saving model 

In [26]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
                   
checkpoint = ModelCheckpoint("FRTL_vgg.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# Note we use a very small learning rate 
modelnew.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

nb_train_samples = 1190
nb_validation_samples = 170
epochs = 3
batch_size = 16

history = modelnew.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

modelnew.save("FRTL_vgg.h5")

Epoch 1/3
74/74 [==============================] - 531s 7s/step - loss: 1.4979 - accuracy: 0.7052 - val_loss: 0.2268 - val_accuracy: 0.7400

Epoch 00001: val_loss improved from inf to 0.22683, saving model to FRTL_vgg.h5
Epoch 2/3
74/74 [==============================] - 504s 7s/step - loss: 0.2461 - accuracy: 0.9062 - val_loss: 0.0176 - val_accuracy: 0.9500

Epoch 00002: val_loss improved from 0.22683 to 0.01758, saving model to FRTL_vgg.h5
Epoch 3/3
74/74 [==============================] - 605s 8s/step - loss: 0.2095 - accuracy: 0.9155 - val_loss: 9.8944e-07 - val_accuracy: 1.0000

Epoch 00003: val_loss improved from 0.01758 to 0.00000, saving model to FRTL_vgg.h5


## Predicting (face recognition)

In [18]:
from PIL import Image
from keras.applications.vgg16 import preprocess_input
import base64
from io import BytesIO
import random
import cv2
from keras.models import load_model
import numpy as np

from keras.preprocessing import image
model = load_model('FRTL_vgg.h5')

# Loading the cascades
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    faces = face_cascade.detectMultiScale(img, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

# Face Recognition with the webcam
video_capture = cv2.VideoCapture(0)
while True:
    _, frame = video_capture.read()
    
    face=face_extractor(frame)
    if type(face) is np.ndarray:
        face = cv2.resize(face, (224, 224))
        im = Image.fromarray(face, 'RGB')
        img_array = np.array(im)
        img_array = np.expand_dims(img_array, axis=0)
        pred = model.predict(img_array)
        print(pred)
        name="None matching"
        
        if(pred[0][0]):
            name='vivek'
        elif(pred[0][1]):
            name="vedant"  
            
        cv2.putText(frame,name, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
    else:
        cv2.putText(frame,"No face found", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
    cv2.imshow('Video', frame)
    if cv2.waitKey(1)==13:
        video_capture.release()
        cv2.destroyAllWindows()

[[0.03114489 0.9688551 ]]
[[0.48964056 0.5103594 ]]
[[0.4070842  0.59291583]]
[[0.09214597 0.9078541 ]]
[[0.6302835  0.36971647]]
[[0.43949586 0.56050414]]
[[0.25098363 0.74901634]]
[[0.35246745 0.6475325 ]]
[[0.7023772 0.2976228]]
[[0.35930732 0.6406928 ]]
[[0.3404071 0.6595928]]
[[0.8993957 0.1006043]]
[[0.8125133  0.18748677]]
[[0.00362631 0.9963737 ]]
[[0.9316208  0.06837926]]
[[0.84071314 0.15928683]]
[[0.02989131 0.9701086 ]]
[[0.11681774 0.8831822 ]]
[[0.18137492 0.81862515]]
[[0.02138075 0.9786193 ]]
[[0.5602846  0.43971533]]
[[0.2390734 0.7609266]]
[[0.18440658 0.81559336]]
[[0.05816208 0.94183785]]
[[0.1567449 0.8432551]]
[[0.9773792  0.02262083]]
[[0.78475267 0.21524735]]
[[0.68039846 0.3196015 ]]
[[0.14309698 0.85690296]]
[[0.47842348 0.5215765 ]]
[[0.350449 0.649551]]
[[0.0394174  0.96058255]]
[[0.09599525 0.90400475]]
[[0.06627991 0.9337201 ]]
[[0.14705506 0.85294497]]
[[0.8318073  0.16819267]]
[[0.09414978 0.90585023]]
[[0.14024998 0.85975   ]]
[[0.02633432 0.9736657 ]]


error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\highgui\src\window.cpp:376: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'
